In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys,os
#import pandas as pd

In [ ]:
hw_path=os.path.abspath('../hw')
if hw_path not in sys.path:
    sys.path.append(hw_path)
from dvl import parse_line

In [ ]:

#data_file='../../data2/230414-121529/viewer_data.pkl'
data_file='../../data2/230427-161356/viewer_data.pkl' #scan
data_file='../../data2/230428-115612/viewer_data.pkl' #grab
data_file='../../data2/230427-153912/viewer_data.pkl' #scan
data_file='../../data2/230427-141001/viewer_data.pkl' #grab
aaa='''
230417-124410,test
230417-130351
230417-131540
230417-134122
230417-134251
230427-123857, in 410 dvl error during scan requiers hanling
230427-125518. grabbing?
230427-133457, scan strange depth, maybe rov out between dives
230427-141001
230427-153912
230427-161241
230427-161356,scan
230427-163312
230427-163859
230428-115612
230428-122153
230428-123852
230428-125713
230428-125908
230428-132522
230428-133824
230428-142729
230428-153056
'''.strip().split('\n')
data_file='../../data2/'+aaa[8].split(',')[0]+'/viewer_data.pkl'
#fd=open(data_file,'rb')
!ls -all -h $data_file
#!ls -1 ../../data2
#!du -s -h ../../data2/*

In [ ]:
ddd={}
with open(data_file,'rb') as fd:
    while 1:
        try:
            d=pickle.load(fd)
            if d[0] not in ddd:
                ddd[d[0]]=[]
            ddd[d[0]].append(d[1:])
        except EOFError:
            break
print(ddd.keys())
    

In [ ]:
start_time=ddd[b'start_time'][0][0]

In [ ]:
ttags=np.array([i[0] for i in ddd[b'thruster_cmd']])-start_time
#ttags=ttags-start_time
cmds=np.array([i[1][1] for i in ddd[b'thruster_cmd']])
print(start_time)

In [ ]:
plt.figure()
ax=plt.subplot(1,1,1)
plt.plot(ttags,cmds[:,:]+np.linspace(0,0.05,8),alpha=0.5)
#plt.plot(ttags,cmds.sum(axis=1),alpha=1)
plt.show()

In [ ]:
ttags=np.array([i[0] for i in ddd[b'topic_att_pitch_control']])
print(ttags[0]-start_time)
#start_time=ttags[0]
ttags=ttags-start_time
N=np.array([i[1]['N'] for i in ddd[b'topic_att_pitch_control']])
T=np.array([i[1]['T'] for i in ddd[b'topic_att_pitch_control']])
R=np.array([i[1]['R'] for i in ddd[b'topic_att_pitch_control']])

#cmds=np.array([i[1][1] for i in ddd[b'topic_att_pitch_control']])

In [ ]:
#ddd[b'topic_att_pitch_control'][:10]
plt.figure()
ax=plt.subplot(1,1,1)
plt.plot(ttags,N,alpha=0.8)
plt.plot(ttags,T,alpha=0.8)
plt.plot(ttags,R,alpha=0.2)
plt.legend(['N-actual','T-target','R-rate'])
#plt.plot(ttags,cmds.sum(axis=1),alpha=1)
plt.show()

In [ ]:
ttags_depth=np.array([i[0] for i in ddd[b'topic_depth_control']])
#start_time=ttags[0]
ttags_depth=ttags_depth-start_time
Nd=np.array([i[1]['N'] for i in ddd[b'topic_depth_control']])
Td=np.array([i[1]['T'] for i in ddd[b'topic_depth_control']])


In [ ]:
#[f for f in ddd[b'topic_tracker']][:3]

In [ ]:
plt.figure('depth control')
ax=plt.subplot(1,1,1)
plt.plot(ttags_depth,Nd,alpha=0.8)
plt.plot(ttags_depth,Td,alpha=0.8)
plt.legend(['N-actual','T-target'])
#plt.plot(ttags,cmds.sum(axis=1),alpha=1)
plt.show()

In [ ]:
ttagsx=np.array([i[0] for i in ddd[b'topic_pos_hold_pid_0']])
ttagsx=ttagsx-start_time
Nx=np.array([i[1]['N'] for i in ddd[b'topic_pos_hold_pid_0']])
Tx=np.array([i[1]['T'] for i in ddd[b'topic_pos_hold_pid_0']])
Rx=np.array([i[1]['R'] for i in ddd[b'topic_pos_hold_pid_0']])

ttagsy=np.array([i[0] for i in ddd[b'topic_pos_hold_pid_1']])
ttagsy=ttagsy-start_time
Ny=np.array([i[1]['N'] for i in ddd[b'topic_pos_hold_pid_1']])
Ty=np.array([i[1]['T'] for i in ddd[b'topic_pos_hold_pid_1']])
Ry=np.array([i[1]['R'] for i in ddd[b'topic_pos_hold_pid_1']])


trange=np.array([i[1]['range'] for i in ddd[b'topic_tracker']])
trange_ttags=np.array([i[0] for i in ddd[b'topic_tracker']])
trange_ttags=trange_ttags-start_time

In [ ]:
plt.figure()
ax=plt.subplot(1,1,1)
plt.plot(ttagsx,Nx,alpha=0.8)
plt.plot(ttagsx,Tx,alpha=0.8)
plt.plot(ttagsx,Rx,alpha=0.2)
plt.plot(trange_ttags,trange,alpha=0.8)
plt.plot(ttagsy,Ny,alpha=0.8)

plt.legend(['N-actual','T-target','R-rate','range','Ny'])
#plt.plot(ttags,cmds.sum(axis=1),alpha=1)
plt.show()

In [ ]:
ts_vel=[]
ts_pos=[]
vel=[]
vel_valid=[]
covs=[]
foms=[]
alts=[]
pos=[]
tr_vels=[[],[],[],[]]
tr_dists=[[],[],[],[]]
tr_rssi=[[],[],[],[]]
tr_nsd=[[],[],[],[]]
tr_ts=[]
yaws=[]
for d in ddd[b'topic_dvl_raw']:
    m=parse_line(d[1]['dvl_raw'])
    if m and m['type'] in ['vel']:#,'deadreacon']:
        ts_vel.append(m['tov']/1e6)
        vel.append((m['vx'],m['vy'],m['vz']))
        covs.append(m['cov'])
        foms.append(m['fom'])
        alts.append(m['alt'])
        vel_valid.append(m['valid']==b'y')
        if m['status']>0:
            print(m)
    if m and m['type'] in ['deadreacon']:
        ts_pos.append(m['time'])
        pos.append((m['x'],m['y'],m['z']))
        yaws.append(m['yaw'])
    if m and m['type'] in ['transducer2']:
        #print(m)
        tr_vels[int(m['id'])].append(m['velocity'])
        tr_dists[int(m['id'])].append(m['distance'])
        tr_rssi[int(m['id'])].append(m['rssi'])
        tr_nsd[int(m['id'])].append(m['nsd'])
        #ts_pos.append(m['time'])
        ts=d[1]['ts']
        if m['id']==0:
            tr_ts.append(ts)
        #pos.append((m['x'],m['y'],m['z']))

def trim(a):
    m=min(map(len,a))
    return [i[:m] for i in a]

ts_vel=np.array(ts_vel)
alts=np.array(alts)
vel=np.array(vel)
covs=np.array(covs)
ts_pos=np.array(ts_pos)
start=ts_pos[0]
ts_pos-=start
ts_vel-=start
pos=np.array(pos)
foms=np.array(foms)
#a1,a2,a3,a4=tr_vels
tr_vels=np.array(trim(tr_vels)).T
tr_dists=np.array(trim(tr_dists)).T
tr_rssi=np.array(trim(tr_rssi)).T
tr_nsd=np.array(trim(tr_nsd)).T
tr_ts=np.array(tr_ts)
_s=tr_ts[0]
tr_ts-=_s
inds=np.nonzero(vel_valid)[0]

In [ ]:
plt.figure('dvl vels')
ax=None
for ind in [0,1,2]: #0 for vx 1 for vy 2 for vz
    ax=plt.subplot(3,1,ind+1,sharex=ax)
    plt.title('v'+'xyz'[ind])
    plt.plot(ts_vel[inds],vel[inds][:,ind],'-b')
#plt.plot(ts_pos,yaws,'+-b')


In [ ]:
plt.figure('dvl alts')
ax=None
ax=plt.subplot(2,1,1,sharex=ax)
plt.plot(ts_vel[inds],alts[inds],'-b')
ax=plt.subplot(2,1,2,sharex=ax)
plt.plot(ttags_depth,Nd,'-g')
plt.show()
#plt.plot(ts_pos,yaws,'+-b')